<h1>Structural Time Series Model with Intervention and Time-Varying Coefficient Data Simulation</h1>

In [3]:
import numpy as np
from scipy.signal import lfilter
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.tools import (
    constrain_stationary_univariate, unconstrain_stationary_univariate)
import sympy as sym
import pandas as pd
import seaborn as sns
import math
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline



In [44]:
# Construct the model
class localtrend(sm.tsa.statespace.MLEModel):
    def __init__(self, endog, x_t):
        k_posdef = 3
        exog = x_t
        # Initialize the state space model
        super(localtrend, self).__init__(endog, exog = exog, k_states=3, k_posdef=3,
                                  initialization='diffuse')
        
        xt = exog[np.newaxis, np.newaxis, :]
        zero = np.zeros([1,1,len(exog)])
        one = np.ones([1,1,len(exog)])
        con_design = np.concatenate((one,zero,xt), axis = 1)

        # Setup the fixed components of the state space representation
        self['design'] = con_design
        self['transition'] = np.eye(self.k_states)
        self['transition', 0, 1] = 1
        self['selection'] = np.eye(self.k_states,k_posdef)
        
        self.positive_parameters = slice(0,4)

    # Describe how parameters enter the model
    def update(self, params, transformed=True, **kwargs):
        params = super(localtrend, self).update(params, transformed, **kwargs)

        self['obs_cov', 0, 0] = params[0]
        self['state_cov', 0, 0] = params[1]
        self['state_cov', 1, 1] = params[2]
        self['state_cov', 2, 2] = params[3]
        

    @property
    def start_params(self):
        exog = sm.add_constant(self.exog)
        res = sm.OLS(self.endog, exog).fit()
        #[np.std(self.endog)]*3
        return np.r_[0.001, 0.001, 0.001, 0.001]
    
    @property
    def param_names(self):
        return ['epsilon','zeta','xi','omega']
    
    def transform_params(self, unconstrained):
        constrained = unconstrained.copy()
        constrained[self.positive_parameters] = (
            constrained[self.positive_parameters] ** 2
        )
        return constrained

    def untransform_params(self, constrained):
        unconstrained = constrained.copy()
        unconstrained[self.positive_parameters] = (
            unconstrained[self.positive_parameters] ** 0.5
        )
        return unconstrained

<li>Local Linear Trend Model<ol>
    <li>Sigma: variance of noise</li>
    <li>Epsilon: disturbance</li>
    <li>Zeta and xi: error </li>
    </ol>
</li>
<br>
    
\begin{equation*}
y_{t} = {\mu}_t + \epsilon_t + \beta_t x_t \quad\epsilon_t \sim N(0,\sigma_{\epsilon}^2)\\
\mu_{t+1} = {\mu}_t + v_t +\xi_t \quad\xi_t \sim N(0,\sigma_{\xi}^2)\\
v_{t+1} = v_t + \zeta_t \quad\zeta_t \sim N(0,\sigma_{\zeta}^2) \\
\beta_{t+1} = \beta_t
\end{equation*}

State Space Representation:
\begin{equation*}
y_{t} = \begin{vmatrix}
1 & 0 & x_t
\end{vmatrix} \begin{pmatrix}
\mu_t \\
v_t \\
\beta_t
\end{pmatrix} + \epsilon_t\\
\begin{pmatrix}
\mu_t \\
v_t \\
\beta_t
\end{pmatrix}= \begin{vmatrix}
1 & 1 & 0\\
0 & 1 & 0\\
0 & 0 & 1
\end{vmatrix} \begin{pmatrix}
\mu_t \\
v_t \\
\beta_t
\end{pmatrix} + \begin{pmatrix}
\xi_t \\
\zeta_t \\
\omega_t
\end{pmatrix}
\end{equation*}


In [45]:
# True model parameters
nobs = int(1e4)
true_epsilon = 9
true_zeta = 1
true_xi = 4
x_var = 0.64
true_omega = 3



In [46]:
# Simulate a time series
np.random.seed(1234)
disturbances_xi = np.random.normal(0, math.sqrt(true_xi), size=(nobs,))
vt = lfilter([1], np.r_[1, -1], disturbances_xi)
disturbances_zeta = np.random.normal(0, math.sqrt(true_zeta), size=(nobs,))
vt_d = vt + disturbances_zeta
mut = lfilter([1], np.r_[1, -1], vt_d)
disturbances_epsilon = np.random.normal(0, math.sqrt(true_epsilon), size=(nobs,))
x_t = np.random.normal(0, math.sqrt(x_var), size=(nobs,))
true_beta = np.cumsum(np.random.normal(0, math.sqrt(true_omega), size=(nobs,)))
intervention = x_t*true_beta
yt = mut + disturbances_epsilon + intervention


In [47]:
mod = localtrend(yt, x_t)
res = mod.fit()
print(res.summary())

                           Statespace Model Results                           
Dep. Variable:                      y   No. Observations:                10000
Model:                     localtrend   Log Likelihood              -33012.551
Date:                Mon, 13 Feb 2023   AIC                          66039.102
Time:                        11:30:29   BIC                          66089.574
Sample:                             0   HQIC                         66056.186
                              - 10000                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
epsilon        8.4586      0.382     22.152      0.000       7.710       9.207
zeta           1.5691      0.779      2.014      0.044       0.042       3.096
xi             4.0208      0.172     23.342      0.0